In [95]:
from datasets import load_dataset
from setfit import SetFitModel, Trainer, TrainingArguments
import evaluate
import numpy as np

In [96]:
import os
print(os.getcwd())

/mnt/c/Users/leonl/OneDrive/College/Senior/CSE 481DS/Analysis


In [97]:
def to_bool(string:str) -> bool:
    string = str(string).strip().casefold()
    if string == 'nan':
        return False
    if string in ('true', 'yes', '1'):
        return True
    if string in ('false', 'no', '0', 'none'):
        return False
    return False

def dsmap(ds, col, fn):
    # I hate huggingface datasets
    return ds.map(lambda x: {**x, **{col: fn(x[col])}})

In [98]:
# Load trainset from ../irr/train_set_gpt/caitlyn.csv
trainset = load_dataset("csv", data_files="./classify/irr/test_set_gpt/caitlyn.csv")["train"]
testset  = load_dataset("csv", data_files="./classify/irr/train_set_human/leon.csv")["train"]

# Set all columns to bool
trainset = dsmap(trainset, 'BreachMentioned', lambda x: int(to_bool(x)))
trainset = dsmap(trainset, 'CompanyMentioned', lambda x: int(bool(x)))
testset  = dsmap(testset, 'BreachMentioned', lambda x: int(to_bool(x)))
testset  = dsmap(testset, 'CompanyMentioned', lambda x: int(bool(x)))

# Remove non-input, non-target columns
trainset = trainset.remove_columns(['Date', 'Publication', 'URL'] + [x for x in trainset.column_names if x.startswith('Unnamed: ')])
testset  = testset.remove_columns(['Date', 'Publication', 'URL'] + [x for x in testset.column_names if x.startswith('Unnamed: ')])

# Split into BreachMentioned and CompanyMentioned
trainset_breach = trainset.remove_columns(['CompanyMentioned'])
testset_breach  = testset.remove_columns(['CompanyMentioned'])

trainset_company = trainset.remove_columns(['BreachMentioned'])
testset_company  = testset.remove_columns(['BreachMentioned'])

In [99]:
model_breach  = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")
model_company = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [108]:
chunk = trainset[105:110]
assert chunk['BreachMentioned'] == trainset_breach[105:110]['BreachMentioned']
assert chunk['CompanyMentioned'] == trainset_company[105:110]['CompanyMentioned']
chunk

{'Headline': ['Data Helps Propel Markets',
  '3 Top D.N.C. Officials Leave in Wake of Email Breach',
  'UN chief claims Syria cease-fire is holding despite growing breaches',
  'Omarosa Manigault Newman releases secret recording of $15,000-a-month job offer from Lara Trump',
  'U.S. Reverses Decision to Classify Data on Afghan Army and PoliceNYT Now'],
 'BreachMentioned': [0, 1, 0, 0, 0],
 'CompanyMentioned': [0, 1, 1, 0, 0]}

In [101]:
args = TrainingArguments(
    batch_size = (16, 32),
    num_epochs = (1, 16),
    output_dir = '/tmp/lleibm/checkpoints',
)

In [102]:
# Train setfit
trainer = Trainer(
    model=model_breach,
    args=args,
    train_dataset=trainset_breach,
    eval_dataset=testset_breach,
    column_mapping={"Headline": "text", "BreachMentioned": "label"},
)

trainer.train()

os.makedirs('./classify/setfit/model_breach', exist_ok=True)
model_breach._save_pretrained('./classify/setfit/model_breach')

metrics = trainer.evaluate()
print(metrics)

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
***** Running training *****
  Num unique pairs = 34312
  Batch size = 16
  Num epochs = 1


KeyboardInterrupt: 

In [ ]:
# Assert that we have both 1 and 0 labels in both
assert 1 in testset_company['CompanyMentioned']

In [ ]:
# Train CompanyMentioned model
# Train setfit
trainer = Trainer(
    model=model_company,
    args=args,
    train_dataset=trainset_company,
    eval_dataset=testset_company,
    column_mapping={"Headline": "text", "CompanyMentioned": "label"},
)

trainer.train()

os.makedirs('./classify/setfit/model_company', exist_ok=True)
model_breach._save_pretrained('./classify/setfit/model_company')

metrics = trainer.evaluate()
print(metrics)

/tmp/ipykernel_3727/3880213943.py:3: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 8000
  Batch size = 16
  Num epochs = 1


Step,Training Loss
1,0.666800
50,0.305400
100,0.184300
150,0.027700
200,0.001500
250,0.000600
300,0.000400
350,0.000400
400,0.000400
450,0.000400


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 0.725}


In [ ]:
# saved_model = SetFitModel._from_pretrained('./classify/setfit/model')

: 

: 

: 

: 